# 밑바닥부터 시작하는 데이터 과학

- 원서명 : Data Science from Scratch: First Principles with Python
- 지은이 : Joel Grus
- 원서 : <http://shop.oreilly.com/product/0636920033400.do>
- 번역서 : <http://www.insightbook.co.kr/books/programming-insight>

![책표지](./image/cover.png)

- 출판사 예제코드 : <https://github.com/insight-book/data-science-from-scratch>

위 책을 보면서 필자가 직접 코딩하면서 정리한 내용입니다.  
책의 모든 내용을 다 포함하고 있지는 않으며, 책에 없는 부가적인 설명이 들어 갈 수 있습니다.  
필자가 작성한 `Jupyter notebook`은 다음 Link에서 다운로드하여 실행이 가능합니다.

- 본문 Jupyter notebook : <https://github.com/DevStarSJ/Study/tree/master/Blog/Python/DataScienceFromScratch>

먼저 이번장에서 사용할 모듈들 및 이전장에서 작성한 함수들을 가져오겠습니다.

In [8]:
import math, random

def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    
    # 무조건 표준정규분포로 검색
    if mu != 0 or sigma !=1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10, 0 # normal_cdf(-10) = 0
    hi_z, hi_p = 10, 1    # normal_cdf(10) = 1
    
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 # 중간값
        mid_p = normal_cdf(mid_z)  # 중간값의 누적분포
        
        if mid_p < p:
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_z, mid_p
        else:
            break
            
    return mid_z

## 07 가설과 추론 (hypothesis and inference)

- `가설(hypothesis)` : 구체적인 주장. (ex. 동전의 앞뒤가 나올 확률은 같다.) 데이터 통계치에 대한 얘기로 변환 될 수 있음

고전적인 가설 검증에서는 `귀무가설(H0, null hypothesis)`와 `대립가설(H1, alternative hypothesis)`로 구성하여 통계를 이용해서 `H0`를 기각할지 말지 결정하는 방법을 사용합니다.

### 1. 예시: 동전 던지기

동전의 앞뒤가 나올 확률이 같다는 것을 검증하기 위해서는 동전의 앞면이 나올 확률 `p=0.5`가 귀무가설이 되고, `p!= 0.5`가 대립가설이 됩니다.

동전을 `n`번 던져서 앞면이 나온 횟수`X`를 세는 것으로 검정을 진행해 보겠습니다.
동전던지기는 `베르누이 분포`를 따를 것이므로, `X`가 이항분포를 따르는 확률변수라는 말이 됩니다.
이항분포는 정규분포에 가깝게 접근합니다.

In [2]:
def normal_approximation_to_binomial(n, p):
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

확률변수가 정규분포를 따른다는 가정하에, `normal_cdf`를 사용하면 실제 동전던지기로부터 얻은 값이 해당 구간에 존재할 확률을 계산할 수 있다.

In [5]:
# 누적분포함수 : 확률변수가 특정 값보다 작을 확률
normal_probability_below = normal_cdf

# 만약 확률변ㅅ후가 특정 값보다 작지 않다면, 크다는 것을 의미
def normal_probability_above(lo, mu=0, sigma=1):
    return 1- normal_cdf(lo, mu, sigma)

# hi와 lo 사이에 존재하는 확률변수 : hi보다 작고, lo보다 큰 범위
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 범위밖에 존재하는 확률변수
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

반대로 확률이 주어졌을 때 평균을 중심으로 하는 대칭적인 구간을 구할 수도 있습니다.
예를 들어 60%를 차지하는 구간을 구하고자 할 경우 양 쪽 부분에서 각각 20%를 차지하는 지점을 구하면 됩니다.

In [9]:
# P(Z <= z) = probability인 z를 return
def normal_upper_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(probability, mu, sigma)

# P(Z >= z) = probability인 z를 return
def normal_lower_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(1 - probability, mu, sigma)

# probability를 중심으로 대칭적인 구간을 return
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    tail_probability = (1 - probability) / 2
    
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

이제 실제로 1000번 던져보겠습니다.

In [4]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

mu_0, sigma_0

(500.0, 15.811388300841896)

X는 대량 평균 500에 표준편차 15.8인 정규분포를 따를것 입니다.

`제1종 오류`를 얼마나 허용해 줄 것인지를 의미하는 `유의수준(significance)`를 결정해야 합니다.
`제1종 오류`란, 비록 H0가 참이지만 H0을 기각하는 `false positive(가양성)` 오류를 의미합니다.
`유의수준`은 통상적으로 5%나 1%로 설정하는 경우가 많은데 여기서는 5%로 해 보겠습니다.

In [10]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

`H0`이 참이라면 (`p = 0.5`) `X`가 주어진 범위를 벗어날 확률은 5%밖에 되지 않아야 합니다. 즉 20번 중 19번은 올바른 결과가 나올것입니다.

`제2종 오류`를 범하지 않을 확률을 구하면 `검정력(power)`을 알 수 있습니다.
`제2종 오류`란 H0가 거짓이지만 H0를 기각하지 않는 오류를 의미합니다.
H0가 거짓이란 것은 어떤 것을 의미할까요 ? 즉, P가 0.5가 아니라는 말 자체는 별 의미가 없으므로, P가 0.55일 확률에 대한 검정력을 구해보겠습니다.